In [1]:
from loader import *
from fastNLP.io import ChnSentiCorpLoader
from fastNLP import DataSet
from fastNLP import Vocabulary
from fastNLP.io import ChnSentiCorpPipe, DataBundle
from spacy.lang.zh import Chinese
import spacy
from fastNLP.embeddings import StaticEmbedding, StackEmbedding
import pandas as pd
from sklearn.utils import shuffle
from sklearn.model_selection import KFold
import numpy as np
from torch import nn
import re

# nlp = Chinese()
# nlp = Chinese.from_config({"nlp": {"tokenizer":{"segmenter": "pkuseg"}}})
# nlp.tokenizer.initialize(pkuseg_model="mixed", pkuseg_user_dict='./data/dict.txt')
from prepare_data import nlp, role_map

In [2]:
doc, q, cho, ans = json_parser('./data/val_fold1.json')
raw_chars = {
        'raw_text': doc,
        'cho'     : cho,
        'q'       : q,
}
ds = DataSet(raw_chars)


In [3]:
doc[0][0]

'<2>反正至少碰面就是會是需要，<2>就是講好的嗎？<1>對。<2>那所以你怎麼決定我這次碰面要吃還是不要吃？<1>就……<1>他如果想要的話，<1>我就會先吃啊。<2>那假設是說，<2>誒不預期這次會不會有可能，<2>就是原本有沒有跟他講好。<1>嗯。<2>通常我們有時候會建議說，<2>如果真的不預期性很大的話，<2>那兩顆你就先吃。<1>嗯。<1>嗯。<2>那如果最後真的沒發生事情……<1>就算了。<2>那你後面兩顆就不用吃嘛。<1>那如果比如說是五六日好了，<1>那我禮拜五吃了嘛，<1>兩顆一顆一顆，<2>對不對？<1>嗯。<2>前面二到兩顆，<2>前面至少兩個小時吃兩顆嘛對不對？<1>嗯。<2>然後你後面這邊下一天再吃一顆，<2>下一天再吃一顆嘛，'

In [4]:
doc[0][0].split('<')

['',
 '2>反正至少碰面就是會是需要，',
 '2>就是講好的嗎？',
 '1>對。',
 '2>那所以你怎麼決定我這次碰面要吃還是不要吃？',
 '1>就……',
 '1>他如果想要的話，',
 '1>我就會先吃啊。',
 '2>那假設是說，',
 '2>誒不預期這次會不會有可能，',
 '2>就是原本有沒有跟他講好。',
 '1>嗯。',
 '2>通常我們有時候會建議說，',
 '2>如果真的不預期性很大的話，',
 '2>那兩顆你就先吃。',
 '1>嗯。',
 '1>嗯。',
 '2>那如果最後真的沒發生事情……',
 '1>就算了。',
 '2>那你後面兩顆就不用吃嘛。',
 '1>那如果比如說是五六日好了，',
 '1>那我禮拜五吃了嘛，',
 '1>兩顆一顆一顆，',
 '2>對不對？',
 '1>嗯。',
 '2>前面二到兩顆，',
 '2>前面至少兩個小時吃兩顆嘛對不對？',
 '1>嗯。',
 '2>然後你後面這邊下一天再吃一顆，',
 '2>下一天再吃一顆嘛，']

In [5]:
ds

+-------------------------------+-------------------------------+-------------------------------+
| raw_text                      | cho                           | q                             |
+-------------------------------+-------------------------------+-------------------------------+
| ['<2>反正至少碰面就是會是...  | ['至少兩個小時吃兩顆', '禮... | 請問醫師建議民眾藥要怎麼吃... |
| ['<2>嗯哼。<1>對，<1>可是...  | ['民眾', '伴侶', '兩個人都... | 下列何者人物是感染者？        |
| ['<2>是這樣吧？<1>嗯。<2>...  | ['吃藥', '抽血', '車禍']      | 請問民眾為什麼上個月開始在... |
| ['<2>概念上是這個樣子啦齁...  | ['方便', '會比較花錢', '要... | 下列何者不是天天吃藥這種吃... |
| ['<2>以前約過的？<1>呵呵呵... | ['有戴保險套', 'APP上約的...  | 下列何者敘述關於民眾最近的... |
| ['<2>最近還好嗎？<1>還好阿... | ['都會帶', '都無套', '一兩... | 請問民眾使用保險套的次數為... |
| ['<4>對，<4>通常，<4>那你...  | ['吃PrEP', '無套', '戴套'...  | 請問個管師建議民眾保護自己... |
| ['<4>你還好嗎？<1>還好。<...  | ['民眾都會戴套', '民眾這個... | 下列敘述何錯誤?               |
| ['<1>最近……<1>最近沒有吃...   | ['匿篩，抽了陰性', '性行為... | 為什麼民眾這幾個月就都沒有... |
| ['<2>有戴套，<2>還是有戴就... | ['第一劑', '第二劑', '第三... | 請問民眾HPV疫苗已施打完第...  |
| ['<2>真的喔

In [6]:
def tokenize(ins):
    role_ins = []
    ins_frag = []
    for frag in ins:
        roleid = []
        tokenize_sen = []
        sents = doc[0][0].split('<')[1:]
        for s in sents:
            sid =s[0]
            # print(s[3:], '\n')
            s = [ele.text for ele in nlp.tokenizer.pipe(s[2:])]
            roleid += [sid] * len(s)
            tokenize_sen+=s

        role_ins.append(roleid)
        ins_frag.append(tokenize_sen)
        assert len(roleid) == len(tokenize_sen)
    return {
        'raw_text': ins_frag,
        'roleid' :role_ins,
    }

In [7]:
bined_query_cho = False


def build_ds(f):
    doc, q, cho, ans = json_parser(f)
    if bined_query_cho :
        cho = [[q[i]+ele for ele in sublist] for i, sublist in enumerate(cho) ]
    raw_chars = {
        'raw_text': doc,
        'cho'     : cho,
        'q'       : q,
    }
    if not ans == []:
        ans= [ele['qa'] for ele in ans] 
        ans_onehot =[np.eye(3)[ele] for ele in ans]
        raw_chars.update({'target':ans, 'target_onehot':ans_onehot})

    ds = DataSet(raw_chars)
    ds.apply_field_more(tokenize, field_name='raw_text', is_input=True,)
    # for k in ['raw_text', 'cho']:
    ds.apply_field(
        lambda ins: [[ele.text for ele in nlp.tokenizer.pipe(sublist)] for sublist in ins], 
        field_name='cho',
        new_field_name='cho', 
        is_input=True
        )
    ds.apply_field(
        lambda ins: [ele.text for ele in nlp.tokenizer.pipe(ins)], 
        field_name='q',
        new_field_name='q', 
        is_input=True
    )    
    if not ans==[]:
        ds.set_input('target_onehot')
        ds.set_target('target') 
    return ds   


ds = {}
train = './data/train_foldidx.json'
val = './data/val_foldidx.json'

for i in range(5):
    ds.update({
        f'train{i}' : build_ds(train.replace('idx', str(i))),
        f'val{i}'   : build_ds(val.replace('idx', str(i))),
    })
ds['dev'] = build_ds('./data/Develop_bined.json')


In [20]:
ds['train3']

+------------------+------------------+------------------+--------+---------------+------------------+
| raw_text         | cho              | q                | target | target_onehot | roleid           |
+------------------+------------------+------------------+--------+---------------+------------------+
| [[82, 53, 45,... | [[176, 902, 2... | [31, 101, 99,... | 0      | [1. 0. 0.]    | [['2', '2', '... |
| [[82, 53, 45,... | [[45, 40, 11,... | [104, 102, 11... | 0      | [1. 0. 0.]    | [['2', '2', '... |
| [[82, 53, 45,... | [[765, 766], ... | [118, 119, 54... | 1      | [0. 1. 0.]    | [['2', '2', '... |
| [[82, 53, 45,... | [[303, 186, 1... | [104, 102, 97... | 2      | [0. 0. 1.]    | [['2', '2', '... |
| [[82, 53, 45,... | [[97, 98, 4, ... | [31, 101, 123... | 2      | [0. 0. 1.]    | [['2', '2', '... |
| [[82, 53, 45,... | [[767, 499, 7... | [31, 101, 168... | 2      | [0. 0. 1.]    | [['2', '2', '... |
| [[82, 53, 45,... | [[668, 943, 7... | [31, 101, 99,... | 2      | [0. 0

In [9]:
raw_key = ['raw_text', 'cho', 'q']

label_ds = DataSet({'target':[0, 1]})
label_vocab  = Vocabulary(unknown=None, padding=None).from_dataset(label_ds, field_name='target')
char_vocab = Vocabulary()
char_vocab.from_dataset(*ds.values(), field_name=raw_key
    ,no_create_entry_dataset=[ds['dev']]
)
char_vocab.index_dataset(*ds.values(), field_name=raw_key)

label_vocab.from_dataset(label_ds, field_name='target')
label_vocab.index_dataset(label_ds, field_name='target')


Vocabulary([0, 1]...)

In [10]:
char_vocab.to_word(0)

'<pad>'

In [11]:
bundle = DataBundle({ 'target':label_vocab}, ds )

In [22]:
set(role_map.values())

{1, 2, 3, 4, 5}

In [30]:
torch.tensor(np.array(['1']).astype(int))

tensor([1])

In [61]:
from torch import nn
from fastNLP.modules import LSTM
import torch
from fastNLP.models.cnn_text_classification import CNNText


class BiLSTMMaxPoolCls(nn.Module):
    def __init__(self, embed, d_model=300, num_layers=1, dropout=0.3):
        super().__init__()
        self.d_model = d_model
        self.embed = embed
        self.num_class = 2 
        self.d_model = d_model
        self.dropout = .2
        num_head = 4
        roleid_size = len(set(role_map.values())) + 1 # pad

        self.role_emb = nn.Embedding(roleid_size, d_model)
        self.emb_enc = EncoderBlock(conv_num=4, d_model=d_model, num_head=num_head, k=7, dropout=0.1)
        self.c_att = nn.MultiheadAttention(d_model, num_head, dropout=self.dropout)
        self.q_att = nn.MultiheadAttention(d_model, num_head, dropout=self.dropout)

        self.lstm = LSTM(d_model, hidden_size=d_model//2, num_layers=num_layers,
                         batch_first=True, bidirectional=True)
        self.bilstm = LSTM(d_model, hidden_size=d_model//2, num_layers=num_layers,
                         batch_first=True, bidirectional=True)
        self.fc = nn.Linear(d_model, self.num_class)


    def forward(self, raw_text, q, cho, roleid, target_onehot=None):  
        PAD = 0
        bsz = raw_text.size(0)
        d_len, q_len, c_len = raw_text.size(-1), q.size(-1), cho.size(-1)

        d_mask = (torch.ones_like(raw_text)*(raw_text==PAD)).bool().view(bsz, 3, d_len)
        q_mask = (torch.ones_like(q)*(q==PAD)).bool().view(bsz, q_len)
        c_mask = (torch.ones_like(cho)*(cho==PAD)).bool().view(bsz, 3, c_len)

        raw_text = raw_text.view(3*bsz, d_len)
        cho = cho.view(3*bsz, c_len)

        
        d_emb = self.embed(raw_text).view(bsz, 3, d_len, self.d_model)
        q_emb = self.embed(q).view(bsz, q_len, self.d_model)
        c_emb = self.embed(cho).view(bsz, 3, c_len, self.d_model)

        role_emb = self.role_emb(torch.from_numpy(roleid.astype(int)).to(device)).view(bsz, 3, d_len, self.d_model)
        # print(role_emb.shape, role_emb.dtype)
        d_emb = d_emb + role_emb

        c = [ele.squeeze(1).transpose(0, 1) for ele in torch.split(c_emb, 1, dim=1)]
        d = [ele.squeeze(1).transpose(0, 1) for ele in torch.split(d_emb, 1, dim=1)]
        d_mask = [ele.squeeze(1) for ele in torch.split(d_mask, 1, dim=1)]
        c_mask = [ele.squeeze(1) for ele in torch.split(c_mask, 1, dim=1)]

        Edc  = [self.c_att(c[i], d[i], d[i], key_padding_mask=d_mask[i])[0] 
                for i in range(3)]
        Eqdc = [self.q_att(q_emb.transpose(0, 1), Edc[i], Edc[i], key_padding_mask=c_mask[i])[0].transpose(0, 1) 
                for i in range(3)]

        Eqdc = torch.stack(Eqdc, dim=1)
        # print(Eqdc.shape)
        Eqdc = Eqdc.view(bsz*3, q_len, self.d_model)

        outputs, _ = self.lstm(Eqdc)
        outputs = outputs[:, -1, :]
        outputs = outputs.view(bsz, 3, self.d_model)
        
        outputs, _ = self.bilstm(outputs)
        outputs = self.fc(outputs)

        if not target_onehot==None:
            loss_fct = nn.CrossEntropyLoss(
                reduction='sum', 
                weight=torch.tensor([0.66, 1.23]).to(outputs),
            )
            outputs = outputs.view(-1, self.num_class)
            # print(outputs.shape, target_onehot.view(-1).shape)
            # print(outputs.dtype)
            loss = loss_fct(outputs, target_onehot.view(-1).to(torch.long)) / bsz
            pred = torch.max(outputs.view(bsz, 3, self.num_class), dim=1).indices[:, 1]
            return {'pred':pred, 'loss':loss} 

        pred = torch.max(outputs.view(bsz, 3, self.num_class), dim=1).indices[:, 1]
        return {'pred':pred} 


In [32]:

from fastNLP.embeddings import BertEmbedding
import torch.nn.functional as F

# embed = StackEmbedding([
#     StaticEmbedding(char_vocab, model_dir_or_name='cn-fasttext'),
#     # StaticEmbedding(char_vocab, model_dir_or_name='cn-sgns-literature-word')
# ])
# embed = BertEmbedding(
#     char_vocab, model_dir_or_name='cn-wwm-ext', layers='1')

# print('Embedding shape', embed.embed_size)

In [65]:
from fastNLP import Trainer
from fastNLP import CrossEntropyLoss
from torch.optim import Adam, AdamW
from fastNLP import AccuracyMetric
from qanet import *
from fastNLP.core.callback import WarmupCallback,GradientClipCallback,EarlyStopCallback,SaveModelCallback
from fastNLP import FitlogCallback
from fastNLP import LRScheduler
from torch.optim.lr_scheduler import LambdaLR
from fastNLP.core import Callback
from fastNLP.core.callback import WarmupCallback,GradientClipCallback,EarlyStopCallback,SaveModelCallback


embed = StackEmbedding([
    StaticEmbedding(char_vocab, model_dir_or_name='cn-fasttext'),
    # StaticEmbedding(char_vocab, model_dir_or_name='cn-sgns-literature-word'),
    # BertEmbedding(char_vocab, model_dir_or_name='cn-wwm-ext', layers='1'),

])

# embed = StackEmbedding([StaticEmbedding(char_vocab, model_dir_or_name='cn-fasttext')])
print('Embedding shape', embed.embed_size)

loss = CrossEntropyLoss()
metric = AccuracyMetric()
device = 0 if torch.cuda.is_available() else 'cpu'

for idx in range(1):
  print(f'--------Fold{idx}----------')

  
  model = BiLSTMMaxPoolCls(embed, d_model=embed.embed_size)
#   model = CNNText(embed, 2, dropout=0.2)
  optimizer = AdamW(model.parameters(), lr=5e-5, weight_decay=0.01,)
  # lrschedule_callback = LRScheduler(lr_scheduler=LambdaLR(optimizer, lambda ep: 1 / (1 + 0.05*ep) ))
  clip_callback = GradientClipCallback(clip_type='value', clip_value=2)
 
  trainer = Trainer(
    train_data=bundle.get_dataset(f'train{idx}'), 
    dev_data=bundle.get_dataset(f'val{idx}'),
    model=model, device=device,
    optimizer=optimizer, batch_size=8, n_epochs=15,
    metrics=metric,
    callbacks=[
      # lrschedule_callback,
      clip_callback,
      WarmupCallback(warmup=50,),
      # EarlyStopCallback(3),
    ]       
  )
  trainer.train()  


Found 1267 out of 1274 words in the pre-training embedding.
Embedding shape 300
--------Fold0----------
input fields after batch(if batch size is 2):
	raw_text: (1)type:torch.Tensor (2)dtype:torch.int64, (3)shape:torch.Size([2, 3, 245]) 
	cho: (1)type:torch.Tensor (2)dtype:torch.int64, (3)shape:torch.Size([2, 3, 7]) 
	q: (1)type:torch.Tensor (2)dtype:torch.int64, (3)shape:torch.Size([2, 21]) 
	target_onehot: (1)type:torch.Tensor (2)dtype:torch.float32, (3)shape:torch.Size([2, 3]) 
	roleid: (1)type:numpy.ndarray (2)dtype:<U1, (3)shape:(2, 3, 245) 
target fields after batch(if batch size is 2):
	target: (1)type:torch.Tensor (2)dtype:torch.int64, (3)shape:torch.Size([2]) 

training epochs started 2021-06-14-16-58-56-547758


  0%|          | 0/1065 [00:00<?, ?it/s, loss:{0:<6.5f}]

  0%|          | 0/17 [00:00<?, ?it/s]

Evaluate data in 0.1 seconds!
Evaluation on dev at Epoch 1/15. Step:71/1065: 
AccuracyMetric: acc=0.423077



  0%|          | 0/17 [00:00<?, ?it/s]

...

Evaluate data in 0.1 seconds!
Evaluation on dev at Epoch 13/15. Step:923/1065: 
AccuracyMetric: acc=0.430769



  0%|          | 0/17 [00:00<?, ?it/s]

Evaluate data in 0.1 seconds!
Evaluation on dev at Epoch 14/15. Step:994/1065: 
AccuracyMetric: acc=0.430769



  0%|          | 0/17 [00:00<?, ?it/s]

Evaluate data in 0.09 seconds!
Evaluation on dev at Epoch 15/15. Step:1065/1065: 
AccuracyMetric: acc=0.430769

Reloaded the best model.

In Epoch:11/Step:781, got best dev performance:
AccuracyMetric: acc=0.430769


In [66]:
from fastNLP.core.predictor import Predictor
pred = Predictor(model).predict(bundle.get_dataset('val0'))['pred']
pred[0:20],pred[20:40],pred[40:60]

([0, 1, 0, 0, 0, 2, 1, 0, 2, 1, 2, 2, 1, 0, 2, 2, 2, 1, 1, 0],
 [1, 2, 2, 1, 2, 0, 2, 2, 0, 1, 0, 1, 0, 0, 2, 0, 0, 2, 2, 2],
 [2, 2, 0, 0, 0, 2, 2, 2, 0, 2, 1, 2, 0, 0, 1, 1, 0, 0, 2, 0])

In [67]:
a = np.array(list(bundle.get_dataset('val0')['target']))
pred = np.array(pred)
np.count_nonzero(a==pred) / a.shape[0]


0.4153846153846154